In [1]:
from __future__ import division, print_function

In [2]:
%load_ext autoreload
%autoreload 2
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
import time
import numpy as np

from director import viewerclient as vc
from irispy import Polyhedron
import boxatlas as box
from contactstabilization import contact_stabilize

In [4]:
vis = vc.Visualizer()

In [5]:
np.sum(np.zeros((2,2)))

0.0

In [21]:
surfaces = [
    box.Surface(Polyhedron.fromBounds([1.2, 0], [1.2, 2]),
            Polyhedron(np.array([[1, -1], [1, 1]]), np.array([0, 0]))),
    box.Surface(Polyhedron.fromBounds([-2, 0], [2, 0]), 
            Polyhedron(np.array([[-2, -1], [2, -1]]), np.array([0, 0]))),
    box.Surface(Polyhedron.fromBounds([-2, 0], [2, 0]), 
            Polyhedron(np.array([[-2, -1], [2, -1]]), np.array([0, 0]))),
    box.Surface(Polyhedron.fromBounds([-1.2, 0], [-1.2, 2]),
            Polyhedron(np.array([[-1, -1], [-1, 1]]), np.array([0, 0]))),
]

env = box.Environment(surfaces, Polyhedron.fromBounds([-1.2, 0], [1.2, 2]))
atlas = box.BoxAtlas()
initial_state = box.BoxAtlasState(atlas)
initial_state.qcom = np.array([0, 1])
initial_state.vcom = np.array([-3, 0.])
initial_state.qlimb = map(np.array, [[0.75, 1], [0.25, 0], [-0.25, 0], [-0.75, 1]])

states, inputs, contact_indicator = contact_stabilize(initial_state, env)

SolutionResult.kSolutionFound


In [31]:
for t in np.linspace(0, states.components[0].breaks[-1] - 0.001, 100):
    box.draw(vis, states(t), inputs(t), env)
    time.sleep(0.05)

In [8]:
i = 0
ts = states.components[0].breaks

In [145]:
box.draw(vis, states(min(ts[i], ts[-1]-0.001)), inputs(min(ts[i], ts[-1]-0.001)), env)
if i < len(ts) - 1:
    i += 1
else:
    i = 0